Here’s a **complete, in-depth note** for the DBMS course topic **Backup & Recovery / 5: Backup / 2: RAID** based on your provided transcript.
I’ve preserved all key concepts, examples, and important formulas while also structuring the material so it’s easy to revise.

---

# **DBMS – Backup & Recovery / 5: Backup / 2: RAID**

## **Learning Outcome**

* Understand **RAID** (Redundant Array of Independent Disks) — its purpose, techniques, and various RAID levels.
* Learn how RAID enhances **speed, reliability, and capacity** for database systems.
* Understand **trade-offs** between performance, cost, and fault tolerance.

---

## **1. Introduction to RAID**

* **RAID** stands for:

  * **Redundant Array of Independent Disks** (current terminology)
  * Originally: **Redundant Array of Inexpensive Disks**
* Goal:

  * Organize **multiple disks in parallel** but **present as a single disk** to the CPU/system.
* Advantages of using multiple disks:

  1. **High Capacity** – Can store larger volumes of data.
  2. **High Speed** – Parallel access increases throughput.
  3. **High Reliability** – Redundancy allows data recovery on disk failure.

---

## **2. Why RAID is Needed**

* Probability of at least one disk failing increases with more disks.
* Example:

  * System with **100 disks**, each with MTTF = 100,000 hours (\~11 years)
  * MTTF of system (any disk failure) ≈ (100,000 hours) / 100 = 1,000 hours (\~41 days)
* RAID **manages failure risk** by:

  * Storing **redundant copies** so recovery is possible even if a disk fails.
* **Important**:

  * Logical redundancy in DB design (bad for normalization) ≠ Physical redundancy in RAID (good for fault tolerance).

---

## **3. Three Primary RAID Techniques**

RAID levels are built by combining these techniques:

### **3.1 Mirroring (Shadowing)**

* Store **duplicate copies** of data.
* **Minimum setup**: 2 physical disks for 1 logical disk.
* **Operation**:

  * **Write** → must be done on both disks.
  * **Read** → can be done from either disk (faster reads).
* **Fault Tolerance**:

  * Data loss only if both original and mirror fail before repair.
  * Handles single disk failure well (except in catastrophic events like fire).
* **Cost**: High storage overhead (50% utilization for 2-disk mirroring).

---

### **3.2 Striping**

* Split data across multiple disks to increase **parallelism**.
* Granularities:

  * **Bit-level striping**:

    * Distribute bits of each byte across disks.
    * High parallel read speed, but high seek time.
    * Rarely used now.
  * **Byte-level striping**:

    * Distribute bytes across disks in a round-robin manner.
    * Improves trade-off between speed and complexity.
    * Less common today.
  * **Block-level striping** (most common):

    * Distribute **blocks** across disks.
    * Requests for different blocks can be served in parallel.
    * Formula: Block *i* → Disk `i mod n + 1`.

---

### **3.3 Parity**

* Add an extra bit/block to detect and correct errors.
* **Odd/Even Parity**:

  * Maintain total number of 1s as odd/even.
  * Detects single-bit errors.
* **Error Recovery**:

  * If disk failure location is known, data can be rebuilt using XOR of bits from remaining disks + parity disk.
* **Parity can be applied** at bit, byte, or block level.
* **Advantage**: Lower storage cost than mirroring.
* **Disadvantage**: Write operations are slower (need to update parity).

---

## **4. RAID Levels**

Combination of above techniques form standard RAID levels.

### **4.1 RAID 0 – Data Striping**

* **Technique**: Block-level striping only (no redundancy).
* **Pros**:

  * Best write & read performance.
  * 100% storage utilization.
  * Cheapest.
* **Cons**:

  * No fault tolerance (failure of 1 disk → total data loss).
* **Use case**: High-performance, temporary data (e.g., real-time rendering).

---

### **4.2 RAID 1 – Mirroring**

* **Technique**: Duplicate data across disks.
* **Pros**:

  * Excellent fault tolerance.
  * Faster reads (parallel from mirrors).
* **Cons**:

  * High cost (50% utilization).
* **Use case**: OS disks, transaction DBs (where reliability is critical).

---

### **4.3 RAID 2 – Bit Striping with Hamming Code**

* **Technique**: Bit-level striping + Hamming error correction.
* **Setup**:

  * *D* data disks + *C* check disks (Hamming bits).
* **Pros**:

  * Detects and corrects single-bit errors; detects double-bit errors.
* **Cons**:

  * Complex; rarely used now.

---

### **4.4 RAID 3 – Byte Striping + Dedicated Parity**

* **Technique**: Byte-level striping + 1 dedicated parity disk.
* **Cons**:

  * Cannot handle multiple requests in parallel (all disks involved in each read/write).
  * Low parallel bandwidth.
* **Use case**: Large sequential data transfers.

---

### **4.5 RAID 4 – Block Striping + Dedicated Parity**

* **Technique**: Block-level striping + 1 dedicated parity disk.
* **Pros**:

  * Allows parallel reads for different blocks.
* **Cons**:

  * Write bottleneck (parity disk updated for every write).
  * Tolerates only 1 disk failure.

---

### **4.6 RAID 5 – Block Striping + Distributed Parity**

* **Technique**: Block-level striping + parity blocks distributed across disks.
* **Pros**:

  * Removes parity bottleneck from RAID 4.
  * High parallelism in reads.
* **Cons**:

  * Only 1 disk failure tolerated.
* **Use case**: Web servers, data warehouses.

---

### **4.7 RAID 6 – Block Striping + Dual Distributed Parity**

* **Technique**: RAID 5 + Second parity block (using Reed-Solomon codes).
* **Pros**:

  * Tolerates 2 disk failures.
* **Cons**:

  * More complex parity calculation.
  * Slower writes.
* **Use case**: High-availability systems with large capacity.

---

## **5. Hybrid RAID Levels**

Combine different RAID levels for improved performance and reliability.

* **RAID 01**: Stripe first (RAID 0), then mirror (RAID 1).

  * Good performance, improved fault tolerance, but low space utilization.
* **RAID 10**: Mirror first (RAID 1), then stripe (RAID 0).

  * Better throughput and latency than RAID 01.
  * Popular for databases.

---

## **6. Choosing the Right RAID Level**

* Factors:

  1. **Performance** (normal, during failure, during rebuild)
  2. **Fault Tolerance** (number of failures tolerated)
  3. **Cost** (capacity utilization)
* **General trends**:

  * **RAID 0** → Fast + cheap, no fault tolerance.
  * **RAID 1** → Fault tolerant, medium performance, high cost.
  * **RAID 5** → Balanced performance, fault tolerance, and cost.
  * **RAID 6** → Extra fault tolerance, lower write performance.
  * **RAID 10** → High performance + fault tolerance, low space efficiency.

---

## **7. RAID vs Backup**

* RAID **is NOT** a replacement for backups.
* RAID protects against:

  * Hardware failures (disk crash, bad sectors)
* RAID does **NOT** protect against:

  * Catastrophic failures (fire, flood)
  * Software bugs
  * Human errors
  * Malware
* Always combine RAID with a **robust backup strategy**.

---

## **8. Key Limitations of RAID**

* Not 100% uptime guarantee.
* Cannot dynamically expand without rebuilding.
* Not always best for virtualization/high-availability clusters (SAN/NAS may be better).

---

## **9. Summary Table**

| RAID Level | Technique                     | Min Disks | Fault Tolerance | Storage Efficiency | Read Performance | Write Performance           | Use Cases                |
| ---------- | ----------------------------- | --------- | --------------- | ------------------ | ---------------- | --------------------------- | ------------------------ |
| **0**      | Block Striping                | 2         | None            | 100%               | High             | High                        | Temp data, rendering     |
| **1**      | Mirroring                     | 2         | 1 disk          | 50%                | High             | Medium                      | OS, critical DBs         |
| **2**      | Bit Striping + Hamming        | ≥3        | 1 disk          | Low                | Medium           | Rarely used                 |                          |
| **3**      | Byte Striping + Parity        | ≥3        | 1 disk          | High               | Low parallelism  | Sequential I/O              |                          |
| **4**      | Block Striping + Parity       | ≥3        | 1 disk          | High               | Medium           | Rarely used                 |                          |
| **5**      | Block Striping + Dist. Parity | ≥3        | 1 disk          | High               | High             | Web/data servers            |                          |
| **6**      | Block Striping + Dual Parity  | ≥4        | 2 disks         | Medium             | Medium           | Archives, high-availability |                          |
| **10**     | Mirroring + Striping          | ≥4        | 1 per mirror    | 50%                | High             | Medium                      | Fast, fault-tolerant DBs |